# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f3f4864e1c0>
├── 'a' --> tensor([[ 0.9942, -0.0981, -0.4941],
│                   [-0.1415,  0.3886,  1.1405]])
└── 'x' --> <FastTreeValue 0x7f3f4864ef10>
    └── 'c' --> tensor([[ 0.9529,  0.6312, -0.7897, -1.3323],
                        [-1.8630,  0.0089, -0.8695, -0.2244],
                        [ 1.3245,  0.9236,  0.7798,  0.8177]])

In [4]:
t.a

tensor([[ 0.9942, -0.0981, -0.4941],
        [-0.1415,  0.3886,  1.1405]])

In [5]:
%timeit t.a

67.7 ns ± 0.0175 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f3f4864e1c0>
├── 'a' --> tensor([[ 1.4581, -0.5823, -1.6445],
│                   [-0.7159, -0.5303,  0.9908]])
└── 'x' --> <FastTreeValue 0x7f3f4864ef10>
    └── 'c' --> tensor([[ 0.9529,  0.6312, -0.7897, -1.3323],
                        [-1.8630,  0.0089, -0.8695, -0.2244],
                        [ 1.3245,  0.9236,  0.7798,  0.8177]])

In [7]:
%timeit t.a = new_value

61.2 ns ± 0.0323 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.9942, -0.0981, -0.4941],
               [-0.1415,  0.3886,  1.1405]]),
    x: Batch(
           c: tensor([[ 0.9529,  0.6312, -0.7897, -1.3323],
                      [-1.8630,  0.0089, -0.8695, -0.2244],
                      [ 1.3245,  0.9236,  0.7798,  0.8177]]),
       ),
)

In [10]:
b.a

tensor([[ 0.9942, -0.0981, -0.4941],
        [-0.1415,  0.3886,  1.1405]])

In [11]:
%timeit b.a

58.8 ns ± 0.0229 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-2.6046,  0.9440,  0.1316],
               [ 1.8215,  1.1776,  0.4019]]),
    x: Batch(
           c: tensor([[ 0.9529,  0.6312, -0.7897, -1.3323],
                      [-1.8630,  0.0089, -0.8695, -0.2244],
                      [ 1.3245,  0.9236,  0.7798,  0.8177]]),
       ),
)

In [13]:
%timeit b.a = new_value

487 ns ± 0.186 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

827 ns ± 0.396 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 50 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

139 µs ± 533 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 771 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f3f485a5910>
├── 'a' --> tensor([[[ 0.9942, -0.0981, -0.4941],
│                    [-0.1415,  0.3886,  1.1405]],
│           
│                   [[ 0.9942, -0.0981, -0.4941],
│                    [-0.1415,  0.3886,  1.1405]],
│           
│                   [[ 0.9942, -0.0981, -0.4941],
│                    [-0.1415,  0.3886,  1.1405]],
│           
│                   [[ 0.9942, -0.0981, -0.4941],
│                    [-0.1415,  0.3886,  1.1405]],
│           
│                   [[ 0.9942, -0.0981, -0.4941],
│                    [-0.1415,  0.3886,  1.1405]],
│           
│                   [[ 0.9942, -0.0981, -0.4941],
│                    [-0.1415,  0.3886,  1.1405]],
│           
│                   [[ 0.9942, -0.0981, -0.4941],
│                    [-0.1415,  0.3886,  1.1405]],
│           
│                   [[ 0.9942, -0.0981, -0.4941],
│                    [-0.1415,  0.3886,  1.1405]]])
└── 'x' --> <FastTreeValue 0x7f3f485e19a0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33 µs ± 94.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f3f48639f70>
├── 'a' --> tensor([[ 0.9942, -0.0981, -0.4941],
│                   [-0.1415,  0.3886,  1.1405],
│                   [ 0.9942, -0.0981, -0.4941],
│                   [-0.1415,  0.3886,  1.1405],
│                   [ 0.9942, -0.0981, -0.4941],
│                   [-0.1415,  0.3886,  1.1405],
│                   [ 0.9942, -0.0981, -0.4941],
│                   [-0.1415,  0.3886,  1.1405],
│                   [ 0.9942, -0.0981, -0.4941],
│                   [-0.1415,  0.3886,  1.1405],
│                   [ 0.9942, -0.0981, -0.4941],
│                   [-0.1415,  0.3886,  1.1405],
│                   [ 0.9942, -0.0981, -0.4941],
│                   [-0.1415,  0.3886,  1.1405],
│                   [ 0.9942, -0.0981, -0.4941],
│                   [-0.1415,  0.3886,  1.1405]])
└── 'x' --> <FastTreeValue 0x7f3e91883ac0>
    └── 'c' --> tensor([[ 0.9529,  0.6312, -0.7897, -1.3323],
                        [-1.8630,  0.0089, -0.8695, -0.2244],
                 

In [23]:
%timeit t_cat(trees)

30.8 µs ± 38.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.5 µs ± 23.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.9529,  0.6312, -0.7897, -1.3323],
                       [-1.8630,  0.0089, -0.8695, -0.2244],
                       [ 1.3245,  0.9236,  0.7798,  0.8177]],
              
                      [[ 0.9529,  0.6312, -0.7897, -1.3323],
                       [-1.8630,  0.0089, -0.8695, -0.2244],
                       [ 1.3245,  0.9236,  0.7798,  0.8177]],
              
                      [[ 0.9529,  0.6312, -0.7897, -1.3323],
                       [-1.8630,  0.0089, -0.8695, -0.2244],
                       [ 1.3245,  0.9236,  0.7798,  0.8177]],
              
                      [[ 0.9529,  0.6312, -0.7897, -1.3323],
                       [-1.8630,  0.0089, -0.8695, -0.2244],
                       [ 1.3245,  0.9236,  0.7798,  0.8177]],
              
                      [[ 0.9529,  0.6312, -0.7897, -1.3323],
                       [-1.8630,  0.0089, -0.8695, -0.2244],
                       [ 1.3245,  0.9236,  0.7798,  0.8177]],

In [26]:
%timeit Batch.stack(batches)

77.5 µs ± 197 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.9529,  0.6312, -0.7897, -1.3323],
                      [-1.8630,  0.0089, -0.8695, -0.2244],
                      [ 1.3245,  0.9236,  0.7798,  0.8177],
                      [ 0.9529,  0.6312, -0.7897, -1.3323],
                      [-1.8630,  0.0089, -0.8695, -0.2244],
                      [ 1.3245,  0.9236,  0.7798,  0.8177],
                      [ 0.9529,  0.6312, -0.7897, -1.3323],
                      [-1.8630,  0.0089, -0.8695, -0.2244],
                      [ 1.3245,  0.9236,  0.7798,  0.8177],
                      [ 0.9529,  0.6312, -0.7897, -1.3323],
                      [-1.8630,  0.0089, -0.8695, -0.2244],
                      [ 1.3245,  0.9236,  0.7798,  0.8177],
                      [ 0.9529,  0.6312, -0.7897, -1.3323],
                      [-1.8630,  0.0089, -0.8695, -0.2244],
                      [ 1.3245,  0.9236,  0.7798,  0.8177],
                      [ 0.9529,  0.6312, -0.7897, -1.3323],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 302 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

309 µs ± 1.61 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
